# Gemini + Chroma で RAG を作ってみる

Langchainを使わないで、
 - ennbedding(ベクトル化)/LLMは、Gemini ennbedding/LLMモデルを直接使用
 - ベクトルDBは、ChromaDB を直接使用

### RAG
1. あらかじめ用意したテキスト(群)を数値化(ベクター化/embedding)し、ベクトルDBに保存
2. テキストに対する質問文を数値化し、数値化されたベクトルDB中の各文書との距離(数値的差)を求め、距離の小さいものの上位を候補として抽出
3. 抽出されたテキストを、質問文とともにLLMへ投げると、質問文に合わせてテキストを解釈し、回答

## まずは簡単なテキストで

Langchain利用に比べると、かなり面倒くさくなる

In [35]:
import google.genai as genai
import chromadb

# ====== Gemini の設定 ======
# Gemini API Keyを取得
with open('GOOGLE_API_KEY.txt', 'r') as f:  # ファイルからAPI Keyを取得
    api_key = f.read().strip()

# Geminiモデルを指定
llm_model       = 'gemini-2.0-flash-exp'
embedding_model = 'gemini-embedding-001'

# Geminiクライアントの作成
gemini_client = genai.Client(api_key=api_key)
# enbedding関数の定義
def embed_texts(texts):
    """
    複数の文字列をGemini Clientで埋め込みベクトルに変換する関数。
        Args: texts (List[str]):
                埋め込み対象の文字列リスト。
        Returns: List[List[float]]:
                各文字列に対応する数値ベクトル（埋め込み）のリスト。
                    [
                      [v11, v12, v13, ...],  # 1つ目の文字列のベクトル
                      [v21, v22, v23, ...],  # 2つ目の文字列のベクトル
                      ...
                    ]
    """
    response = gemini_client.models.embed_content(
        model=embedding_model,
        contents=texts
    )
    return [e.values for e in response.embeddings]

# ====== ChromaDB の設定 ======
# clientの作成
# 　メモリ上だけ (永続化しない), 
# 　永続化させる場合には、chromadb.PersistentClient(path='save_to')を使用)
chroma_client = client = chromadb.EphemeralClient()

# DB内collectionの初期化(既存のcollectionがあったら削除)
collection_name = 'novels'   # 任意の文字列
try:
    chroma_client.delete_collection(collection_name)
except:
    pass
collection = chroma_client.create_collection(collection_name)

In [50]:
# 検索対象テキスト
texts = [
    '石川です。エンジニアです。散歩は嫌いではありません。', 
    '果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。',
    '佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。',
    '東京の渡辺さんと佐藤さんは、たまに居酒屋で一緒に飲んでいるようです。',
    'サイクリングの好きな鈴木さんは自転車で会社へ通勤しています。高橋さんは会社の同僚です。',
    '伊藤さんと山本さんは、よく一緒に奥多摩へキャンプに行くようです',
    '中村さんは高橋さんの勤めている会社の上司です。よく飲みに誘われますが参加するかは半々です。',
    '京都にお住いの小林さんは、佐藤さんのおいです。',
    '加藤さんは5人家族です。',
]

# テキストをエンベディング化(vector化)する
embeddings = embed_texts(texts)# => [[vec1], [vec2], ...]

# 確認: テキストのエンベディング内容(先頭の2テキスト、enbeddingの先頭の5次元分)
print(f"len(embeddings)={len(embeddings)}")
for i, embedding in enumerate(embeddings[:2]):
    print(f"{i:2d} embedding First 5 values: {embedding[:5]}")
# -------------------------------------------

# Document から必要な情報をまとめる
ids = [f"doc{i}" for i in range(len(texts))] # id: ユニークな文字列

# 確認
print(f"Confirm: len(ids)={len(ids)}, len(embeddings)={len(embeddings)}, len(texts)={len(texts)}")

# 一括で追加
collection.add(
    ids=ids,
    embeddings=embeddings,
    documents=texts,
#    metadatas=metadatas        # (今回未使用、空辞書だとエラー)
)
print(f"collection count={collection.count()}")

# 確認: DB検索 -------------------------------
query = '山田さんの職業は何ですか。'
results = collection.query(
    query_embeddings=embed_texts(query), # queryのリスト(複数のqueryを投げられる)をベクトル化
    n_results=3,                           # 上位の取得件数
#    where={"author": "夏目漱石"}           # ★ metadata フィルタ(今回未使用)
)
retreaved_texts = results['documents'][0]

print(f"test ChromaDB: Question={query}")
for i in range(len(results['documents'][0])):     # [0]が必要なのは、複数のqueryを投げられるため
    print(f"retrieved doc {i}: id={results['ids'][0][i]}, " +
          f"distances={results['distances'][0][i]}, metadata={results['metadatas'][0][i]}" +
          f"\n          text={results['documents'][0][i][:200]}")   # 最初の200文字だけ表示
# -------------------------------------------

# LLMモデルを使用してレスポンスを生成
queries = []
queries.append('山田さんの職業は何ですか。')
queries.append('佐藤さんの職業は何ですか。')
queries.append('お酒を飲む人は誰ですか。')
queries.append('関東に住んでいる人は誰ですか。')
print(f"\nQ&A ----------------")
for q_no, query in enumerate(queries):
    results = collection.query(
        query_embeddings=embed_texts([query]),
        n_results=3
    )
    retreaved_texts = results['documents'][0]
    response = gemini_client.models.generate_content(
        model=llm_model,
        contents=[f"QUESTION{query}", f"CONTENTS"] + retreaved_texts,
        config=genai.types.GenerateContentConfig(
            system_instruction="あなたは気さくな隣人です。質問に日常会話的に答えてください。",
            temperature=0.7,  # ★ creativity の度合いを調整
        )
    )
    print(f"\nQ{q_no}: query={query},\nresponse={response.text}")
    for i in range(len(results['documents'][0])):     # [0]が必要なのは、複数のqueryを投げられるため
        print(f"retrieved doc {i}: id={results['ids'][0][i]}, " +
              f"distances={results['distances'][0][i]}, metadata={results['metadatas'][0][i]}" +
              f"\n          text={results['documents'][0][i][:200]}")   # 最初の200文字だけ表示


len(embeddings)=9
 0 embedding First 5 values: [0.0010123871, -0.027725141, 0.004763818, -0.085715435, -0.016764862]
 1 embedding First 5 values: [-0.0032862844, -0.019554267, 0.006109047, -0.07946284, 0.008544021]
Confirm: len(ids)=9, len(embeddings)=9, len(texts)=9
collection count=9
test ChromaDB: Question=山田さんの職業は何ですか。
retrieved doc 0: id=doc1, distances=0.4490359425544739, metadata=None
          text=果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。
retrieved doc 1: id=doc4, distances=0.732514500617981, metadata=None
          text=サイクリングの好きな鈴木さんは自転車で会社へ通勤しています。高橋さんは会社の同僚です。
retrieved doc 2: id=doc2, distances=0.743468165397644, metadata=None
          text=佐藤さんは酒屋を営んでいました。現在はコンビニの店長です。泣き上戸です。

Q&A ----------------

Q0: query=山田さんの職業は何ですか。,
response=あら、山田さんは果物屋さんなんですね！前は八百屋さんだったんですか。甘いものが好きで、お酒は飲めないって、なんだか親近感湧きますね！

retrieved doc 0: id=doc1, distances=0.4490359425544739, metadata=None
          text=果物屋の山田です。以前は八百屋でした。甘いものが好きです。下戸です。
retrieved doc 1: id=doc4, distances=0.732514500617981, metada

## 大きなドキュメントの分割、メタデータの活用(未完)

ドキュメントとして、青空文庫を利用させていただきました。

「Langchainを使わないで、」という条件は日和ってしまいました。

以下の機能は単機能で、特にRecursiveCharacterTextSplitter()は代わりになるライブラリが見つからず、スクラッチで作成するのも結構大変ということで、使わせていただきます。
 - Documentクラス                    # テキスト+メタデータ  
 - DirectoryLoader()                 # ディレクトリ単位でファイルを取得、Document型へ  
 - RecursiveCharacterTextSplitter()  # Document型のテキストを指定サイズでChankへ分割  
                                     # 区切文字列、ラップサイズを指定、メタデータを各Chankへコピー  


In [ ]:
import google.genai as genai
import chromadb
from langchain.schema import Document
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# テキストファイル・メタデータ
matadata_dic = {
    '1_kumono_ito.txt': {
        'genre': 'novel', 'title':'蜘蛛の糸',          'author':'芥川竜之介', 'year':'1918'},
    '2_chumonno_oi_ryoriten.txt': {
        'genre': 'novel', 'title':'注文の多い料理店',  'author':'宮沢賢治',   'year':'1924'},
    '2_kazeno_matasaburo.txt': {
        'genre': 'novel', 'title':'風の又三郎',        'author':'宮沢賢治',   'year':'1934'},
    '2_serohikino_goshu.txt': {
        'genre': 'novel', 'title':'セロ弾きのゴーシュ', 'author':'宮沢賢治',   'year':'1934'},
    '2_gingatetsudono_yoru.txt': {
        'genre': 'novel', 'title':'銀河鉄道の夜',      'author':'宮沢賢治',   'year':'1934'},
    '3_bocchan.txt': {
        'genre': 'novel', 'title':'坊ちゃん',         'author':'夏目漱石',   'year':'1906'},
    '4_wagahaiwa_nekodearu.txt': {
        'genre': 'novel', 'title':'吾輩は猫である',    'author':'夏目漱石',   'year':'1905'},
    '4_kaijin_nijumenso.txt': {
        'genre': 'novel', 'title':'怪人二十面相',      'author':'江戸川乱歩', 'year':'1936'},
    '5_sanshodayu.txt': {
        'genre': 'novel', 'title':'山椒大夫',         'author':'森鷗外',     'year':'1915'},
}
null_dic = {'genre':'','title':'', 'author':'','year':''}

# 小説データをロード
novels_dir = './novels/'
loader = DirectoryLoader(novels_dir, glob='*.txt') # ディレクトリ内の.txtを指定して
documents = loader.load()   # ドキュメント(メタデータ(ファイル名)+テキスト)としてロード

# 各ドキュメントにメタデータを付与
for doc in documents:
    fn = os.path.basename(doc.metadata["source"])
    doc.metadata.update({'filename': fn} | matadata_dic.get(fn, null_dic))
    print(doc.metadata) # 確認

# ドキュメントをチャンクへ分割
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=10000,
    chunk_overlap=100,
    separators=['。', '\n'] # 分割位置の指定
)
chunks = text_splitter.split_documents(documents)

